In [1]:
# Imports
import numpy as np 
import nibabel as nib
import os
import torch
import monai
import shutil

from scipy.ndimage import binary_dilation
from scipy.ndimage import generate_binary_structure


c:\Users\P095550\Documents\CRLM-morph-features\CRLM-morph-features\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Tumor mask data preparation

In [2]:
# All paths 
all_scans_path = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/all_scans"
all_segmentations_path = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/all_segmentations"

tumor_segmentations_path = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/tumor_segmentations"
paired_tumor_scans_path = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/paired_tumor_scans"

In [6]:
"""
Segments the liver, creates a bounding box based on the liver, and segments only the tumors within the bounding box.
"""

struct = np.ones((3, 3, 3))  # 3x3x3 cube

# Generates a 3D spherical-like connectivity structure
structure = generate_binary_structure(3, 1)  # 3D, with connectivity=1

for scan in os.listdir(all_scans_path):
    print(f"Currently processing: {scan}")

    new_filename = (scan.split(".")[0])[0:-5] + "_tumor.nii.gz"
    if new_filename not in os.listdir(tumor_segmentations_path):
        # Load image and corresponding segmentation
        image = nib.load(os.path.join(all_scans_path, scan))

        segm_filename = scan.replace("_0000", "")
        segmentation = nib.load(os.path.join(all_segmentations_path, segm_filename))

        image_data = image.get_fdata()
        segmentation_data = segmentation.get_fdata()

        # Create a liver mask (labels 12 and 13)
        liver_mask = (segmentation_data == 12) | (segmentation_data == 13)

        # Apply the liver mask to the image
        liver_image = np.copy(image_data)
        liver_image[~liver_mask] = -1000

        # Find the indices of the liver mask
        mask_indices = np.argwhere(liver_mask)

        # Calculate the bounding box for the liver
        min_indices = mask_indices.min(axis=0)
        max_indices = mask_indices.max(axis=0)

        # Crop the liver image using the bounding box
        cropped_liver_image = liver_image[min_indices[0]:max_indices[0]+1, min_indices[1]:max_indices[1]+1, min_indices[2]:max_indices[2]+1]

        # Crop the segmentation data using the same bounding box
        cropped_segmentation_data = segmentation_data[min_indices[0]:max_indices[0]+1, min_indices[1]:max_indices[1]+1, min_indices[2]:max_indices[2]+1]

        # Create a tumor mask (label 13) within the cropped liver region
        tumor_mask = (cropped_segmentation_data == 13)

        # Apply binary dilation to the tumor mask
        dilated_tumor_mask = binary_dilation(tumor_mask, structure=structure, iterations=10)

        # Apply the dilated tumor mask to the cropped liver image
        tumor_image = np.copy(cropped_liver_image)
        tumor_image[~dilated_tumor_mask.astype(bool)] = -1000

        # Create a new NIfTI image for the tumor segmentation
        new_image = nib.Nifti1Image(tumor_image, affine=image.affine, header=image.header)

        # Save the new NIfTI image to a file with the original name
        
        output_file_path = os.path.join(tumor_segmentations_path, new_filename)
        nib.save(new_image, output_file_path)

Currently processing: CAESAR326_1_0000.nii.gz
Currently processing: CAESAR187_0_0000.nii.gz
Currently processing: CAESAR360_1_0000.nii.gz
Currently processing: CAESAR357_1_0000.nii.gz
Currently processing: CAESAR012_1_0000.nii.gz
Currently processing: CAESAR429_1_0000.nii.gz
Currently processing: CAESAR453_1_0000.nii.gz
Currently processing: CAESAR442_0_0000.nii.gz
Currently processing: CAESAR569_1_0000.nii.gz
Currently processing: CAESAR257_2_0000.nii.gz
Currently processing: CAESAR226_0_0000.nii.gz
Currently processing: CAESAR583_0_0000.nii.gz
Currently processing: CAESAR419_0_0000.nii.gz
Currently processing: CAESAR537_0_0000.nii.gz
Currently processing: CAESAR269_1_0000.nii.gz
Currently processing: CAESAR372_0_0000.nii.gz
Currently processing: CAESAR404_1_0000.nii.gz
Currently processing: CAESAR512_1_0000.nii.gz
Currently processing: CAESAR417_1_0000.nii.gz
Currently processing: CAESAR001_1_0000.nii.gz
Currently processing: CAESAR299_0_0000.nii.gz
Currently processing: CAESAR463_2_

In [7]:
"""
copies scans of patients that have both 0 and 1 scans
"""

# Create a dictionary to track patient scans
patient_scans = {}

# Iterate through all scans
for scan in os.listdir(tumor_segmentations_path):
    # Extract the patient ID and scan type (0 or 1)

    patient_id, scan_type = scan.split("_")[0], scan[0:-7].split("_")[1]

    # Add the scan type to the patient's record
    if patient_id not in patient_scans:
        patient_scans[patient_id] = set()
    patient_scans[patient_id].add(scan_type)

# Move scans of patients that have both 0 and 1 scans
for patient_id, scan_types in patient_scans.items():
    if "0" in scan_types and "1" in scan_types:
        first_scan = f"{patient_id}_0_tumor.nii.gz"
        second_scan = f"{patient_id}_1_tumor.nii.gz"
        shutil.copy(src=os.path.join(tumor_segmentations_path, first_scan), dst=os.path.join(paired_tumor_scans_path, first_scan))
        shutil.copy(src=os.path.join(tumor_segmentations_path, second_scan), dst=os.path.join(paired_tumor_scans_path, second_scan))

## Weighted tumor mask data preparation

### Sanity checks

In [41]:
# Check the number of slices for each scan in the paired_tumor_scans folder
for scan in os.listdir(paired_tumor_scans_path):
    scan_path = os.path.join(paired_tumor_scans_path, scan)
    nifti_image = nib.load(scan_path)
    num_slices = nifti_image.shape[2]  # Assuming slices are along the third dimension
    print(f"{scan}: {num_slices} slices")

CAESAR550_1_tumor.nii.gz: 114 slices
CAESAR500_1_tumor.nii.gz: 60 slices
CAESAR455_0_tumor.nii.gz: 65 slices
CAESAR072_0_tumor.nii.gz: 34 slices
CAESAR339_1_tumor.nii.gz: 54 slices
CAESAR438_1_tumor.nii.gz: 30 slices
CAESAR410_0_tumor.nii.gz: 56 slices
CAESAR047_1_tumor.nii.gz: 27 slices
CAESAR213_1_tumor.nii.gz: 53 slices
CAESAR338_0_tumor.nii.gz: 69 slices
CAESAR449_0_tumor.nii.gz: 36 slices
CAESAR358_1_tumor.nii.gz: 44 slices
CAESAR148_1_tumor.nii.gz: 44 slices
CAESAR213_0_tumor.nii.gz: 250 slices
CAESAR129_0_tumor.nii.gz: 106 slices
CAESAR402_0_tumor.nii.gz: 70 slices
CAESAR083_1_tumor.nii.gz: 38 slices
CAESAR110_0_tumor.nii.gz: 32 slices
CAESAR044_0_tumor.nii.gz: 31 slices
CAESAR110_1_tumor.nii.gz: 31 slices
CAESAR139_0_tumor.nii.gz: 75 slices
CAESAR547_0_tumor.nii.gz: 60 slices
CAESAR051_0_tumor.nii.gz: 165 slices
CAESAR242_0_tumor.nii.gz: 73 slices
CAESAR012_1_tumor.nii.gz: 47 slices
CAESAR415_0_tumor.nii.gz: 62 slices
CAESAR473_1_tumor.nii.gz: 52 slices
CAESAR323_1_tumor.nii.gz

In [ ]:
# check with resizing if segmentations are still correct
